In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

# Configure plotting
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_theme(style="whitegrid")

# Ensure reports directory exists
os.makedirs('../reports', exist_ok=True)

# 1. Load Data
# --------------------------------------------------------------------------------
file_path = '../data/raw/ethiopia_fi_unified_data.xlsx'
print(f"Loading data from {file_path}...")

try:
    xl = pd.ExcelFile(file_path)
    # Load Main Data and Impact Links
    df_data = xl.parse('ethiopia_fi_unified_data')
    df_impact = xl.parse('Impact_sheet')
    
    print(f"Data shape: {df_data.shape}")
    print(f"Impact links shape: {df_impact.shape}")
    
except Exception as e:
    print(f"Error loading data: {e}")
    raise

# 2. Preprocess Events
# --------------------------------------------------------------------------------
# Filter for events: record_type == 'event'
events_df = df_data[df_data['record_type'].str.lower() == 'event'].copy()
print(f"Found {len(events_df)} events.")

# Ensure date column is datetime
events_df['date'] = pd.to_datetime(events_df['observation_date'])

# Handle Names: The 'indicator' column contains the Event Name (e.g. Telebirr Launch)
# 'value_text' contains status like 'Launched'.
events_df['name'] = events_df['indicator'].fillna(events_df['value_text']).fillna(events_df['record_id'])

# Display a few events
print("Sample Events:")
print(events_df[['name', 'date', 'record_id']].head())

# 3. Join Events with Impact Links
# --------------------------------------------------------------------------------
# Join on parent_id (in impact_links) -> record_id (in events)

merged_impacts = pd.merge(
    df_impact, 
    events_df[['record_id', 'name', 'date']], 
    left_on='parent_id', 
    right_on='record_id', 
    how='inner',
    suffixes=('_link', '_event')
)

print(f"\nMerged {len(merged_impacts)} impact relationships.")
print("Unique Link Indicator Codes:", df_impact['indicator_code'].unique())
print("Unique Link Related Indicator:", df_impact['related_indicator'].unique())

# Use related_indicator if indicator_code is empty
if df_impact['indicator_code'].isna().all():
    print("Switching to related_indicator as indicator_code is empty.")
    merged_impacts['indicator_code'] = merged_impacts['related_indicator']
else:
    print("Using indicator_code.")

print("Merged Indicator Codes:", merged_impacts['indicator_code'].unique())
print(merged_impacts[['parent_id', 'name', 'indicator_code', 'impact_direction']].head())

# 4. Impact Summary & Matrix
# --------------------------------------------------------------------------------
def parse_effect(row):
    # impact_direction: Positive/Negative
    direction = str(row['impact_direction']).lower()
    sign = 1 if 'positive' in direction else (-1 if 'negative' in direction else 0)
    
    # impact_magnitude: High/Medium/Low or numeric
    mag_text = str(row['impact_magnitude']).lower()
    if 'high' in mag_text: mag = 3
    elif 'medium' in mag_text: mag = 2
    elif 'low' in mag_text: mag = 1
    else: mag = 1 # Default
    
    return sign * mag

merged_impacts['impact_score'] = merged_impacts.apply(parse_effect, axis=1)

# Check for duplicates before pivot (Sum if multiple entries exist)
impact_matrix = merged_impacts.pivot_table(
    index='name', 
    columns='indicator_code', 
    values='impact_score', 
    aggfunc='sum'
).fillna(0)

print("\nEvent-Indicator Impact Matrix:")
print(impact_matrix)

# Save Matrix
impact_matrix.to_csv('../reports/impact_matrix.csv')
print("\nSaved impact matrix to ../reports/impact_matrix.csv")

# 5. Model Verification (Telebirr Case Study)
# --------------------------------------------------------------------------------
target_event_keyword = "Telebirr"
target_indicator = "ACC_MM_ACCOUNT"

print(f"\n--- Validation Case: {target_event_keyword} Impact on {target_indicator} ---")

# Find the specific event
telebirr_events = events_df[events_df['name'].str.contains(target_event_keyword, case=False, na=False)]

if not telebirr_events.empty:
    event_date = telebirr_events.iloc[0]['date']
    event_name = telebirr_events.iloc[0]['name']
    print(f"Found Event: {event_name} on {event_date.date()}")
    
    # Get Time Series Data for the indicator
    # Note: data sheet has 'record_type' for indicators too? Or lines where indicator_code is set.
    # Let's filter by indicator_code.
    indicator_data = df_data[df_data['indicator_code'] == target_indicator].copy()
    
    # If empty, maybe check 'indicator' column?
    if indicator_data.empty:
         print(f"Searching for indicator by name text: 'Mobile Money'...")
         indicator_data = df_data[df_data['indicator'].str.contains('Mobile Money', case=False, na=False)].copy()

    indicator_data['date'] = pd.to_datetime(indicator_data['observation_date'])
    indicator_data['value'] = pd.to_numeric(indicator_data['value_numeric'], errors='coerce')
    indicator_data = indicator_data.dropna(subset=['value', 'date']).sort_values('date')
    
    if not indicator_data.empty:
        print(f"Found {len(indicator_data)} data points for {target_indicator}")
        # Plot
        plt.figure(figsize=(10, 5))
        plt.plot(indicator_data['date'].values, indicator_data['value'].values, marker='o', label='Actual Data')
        plt.axvline(x=event_date, color='r', linestyle='--', label=f'{event_name} Launch')
        plt.title(f"Impact of {event_name} on {target_indicator}")
        plt.xlabel("Year")
        plt.ylabel("Value (%)")
        plt.legend()
        plt.grid(True)
        plt.savefig('../reports/telebirr_validation.png')
        print("Saved validation plot to ../reports/telebirr_validation.png")
        
        # Check impact direction logic
        pre_data = indicator_data[indicator_data['date'] < event_date]['value'].mean()
        post_data = indicator_data[indicator_data['date'] > event_date]['value'].mean()
        
        print(f"Pre-Event Mean: {pre_data:.2f}")
        print(f"Post-Event Mean: {post_data:.2f}")
        
    else:
        print(f"No numeric data found for indicator {target_indicator}")

else:
    print(f"Warning: Event containing '{target_event_keyword}' not found in data.")
